## Ayush Srivastava, MDS202113
### Applied Machine Learning
### Assignment 2


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importing Libraries**



In [39]:
%matplotlib inline
import mlflow
import logging
import pandas as pd
import numpy as np
from urllib.parse import urlparse
from markupsafe import escape
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,precision_recall_curve,auc
from sklearn.ensemble import RandomForestClassifier

In [9]:
! pip install mlflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 KB 8.2 MB/s eta 0:00:00
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.0.3
    Uninstalling Jinja2-3.0.3:
      Successfully uninstalled Jinja2-3.0.3


In [10]:
! pip install jinja2==3.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.0.3 which is incompatible.


In [11]:
import pandas as pd

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
!pip install textblob
from sklearn.model_selection import GridSearchCV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
%matplotlib inline
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
import regex as re
from nltk.stem import WordNetLemmatizer
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

In [22]:
import regex as re

In [23]:
raw_data = pd.read_csv('/content/drive/MyDrive/smsspamcollection/SMSSpamCollection',sep='\t',header=None)
raw_data.columns=['Label','Text']
raw_data.head()

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
raw_data.isna().sum()

Label    0
Text     0
dtype: int64

In [25]:
raw_data.groupby('Label').describe()

Text                                                               
      count unique                                                top freq
Label                                                                     
ham    4825   4516                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [26]:
raw_data.loc[raw_data.Label == 'spam', 'Label'] = 1
raw_data.loc[raw_data.Label == 'ham', 'Label'] = 0

In [27]:
raw_data.head()

,Label,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
train, validate, test =  np.split(raw_data.sample(frac=1, random_state=42), 
                       [int(.6*len(raw_data)), int(.8*len(raw_data))])


In [33]:
X_train= train.Text
Y_train=train.Label

**Loading Training data, Validation data and Test data**

In [32]:
X_validate = validate.Text
Y_validate = validate.Label
X_test = test.Text
Y_test = test.Label


In [34]:
count = CountVectorizer().fit(raw_data.Text)

In [35]:
X_train = count.transform(X_train)
X_val = count.transform(X_validate)
X_test = count.transform(X_test)

In [36]:
tfidf_transformer = TfidfTransformer()
tfidf_train = tfidf_transformer.fit_transform(X_train)
tfidf_val = tfidf_transformer.fit_transform(X_val)
tfidf_test = tfidf_transformer.fit_transform(X_test)

In [47]:
Y_train=Y_train.astype('int')
Y_validate=Y_validate.astype('int')
Y_test=Y_test.astype('int')

**Model 1:  Random Forest Model** 

In [38]:
def eval_metrics(actual, pred):
    precision, recall, thresholds = precision_recall_curve(actual, pred)
    auc_precision_recall = auc(recall, precision)
    return (auc_precision_recall)

In [48]:
mlflow.sklearn.autolog()

n_estimators = 200
max_depth = 5
clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth,random_state=101)
Y_train=Y_train.astype('int')
clf.fit(tfidf_train, Y_train)

y_pred = clf.predict(tfidf_test)
aucpr = eval_metrics(Y_test, y_pred)
acc = accuracy_score(Y_test, y_pred)
conf_1=confusion_matrix(Y_test, y_pred)

with mlflow.start_run(run_name=f"n_estimators : {n_estimators}, max_depth : {max_depth}"):
  
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.log_dict(np.array(conf_1).tolist(), "confusion_matrix.json")
    mlflow.sklearn.log_model(clf, "model")

    print("\nRandom Classifier Model (no_of_estimator={:f}, max_depth={:f}):".format(n_estimators, max_depth))
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr} ")
    print(f"Confusion Matrix: {conf_1} \n \n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="random-forest-classification-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="Randomclassifier")
    else:
      mlflow.sklearn.log_model(clf, "model")

2023/02/27 09:53:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cfddf65150834d76aae1c2e1fef80750', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Random Classifier Model (no_of_estimator=200.000000, max_depth=5.000000):
Accuracy: 0.863677130044843
AUCPR: 0.574654941471085 
Confusion Matrix: [[961   0]
 [152   2]] 
 



Successfully registered model 'random-forest-classification-model'.
2023/02/27 09:54:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-classification-model, version 1
Created version '1' of model 'random-forest-classification-model'.


In [49]:
mlflow.tracking.MlflowClient().get_model_version("random-forest-classification-model","1")

<ModelVersion: creation_timestamp=1677491641268, current_stage='None', description=None, last_updated_timestamp=1677491641268, name='random-forest-classification-model', run_id='5036bdb314754812b9fbd4c96ad143eb', run_link=None, source='file:///content/mlruns/0/5036bdb314754812b9fbd4c96ad143eb/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>

**Model 2: Multinomial Naive Bayes**

In [51]:
mlflow.sklearn.autolog()

clf = MultinomialNB()
clf.fit(tfidf_train, Y_train)

y_pred = clf.predict(tfidf_test)
aucpr = eval_metrics(Y_test, y_pred)
acc = accuracy_score(Y_test, y_pred)
conf_2=confusion_matrix(Y_test, y_pred)

with mlflow.start_run(run_name="Multinomial Naive Bayes"):

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.sklearn.log_model(clf, "model")
    mlflow.log_dict(np.array(conf_2).tolist(), "confusion_matrix.json")

    print("\nMultinomial Naive Bayes")
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr}")
    print(f"Confusion Matrix: {conf_2} \n\n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="multinomial-nb-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="multinomial-nb-model")
    else:
      mlflow.sklearn.log_model(clf, "model")

2023/02/27 09:57:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '55b62ea0372f42aca0b83383f59e0c52', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Multinomial Naive Bayes
Accuracy: 0.9390134529147982
AUCPR: 0.8097140527633802
Confusion Matrix: [[961   0]
 [ 68  86]] 




Successfully registered model 'multinomial-nb-model'.
2023/02/27 09:57:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: multinomial-nb-model, version 1
Created version '1' of model 'multinomial-nb-model'.


In [52]:
print(mlflow.tracking.MlflowClient().get_model_version("multinomial-nb-model", '1'))

<ModelVersion: creation_timestamp=1677491878100, current_stage='None', description=None, last_updated_timestamp=1677491878100, name='multinomial-nb-model', run_id='37b2550b2e36402ca4aed4f47002a3b3', run_link=None, source='file:///content/mlruns/0/37b2550b2e36402ca4aed4f47002a3b3/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>


**Model 3: MLP Classifier**

In [55]:
mlflow.sklearn.autolog()

clf = MLPClassifier(random_state=101,learning_rate='adaptive')
clf.fit(tfidf_train, Y_train)

y_pred = clf.predict(tfidf_test)
aucpr = eval_metrics(Y_test, y_pred)
acc = accuracy_score(Y_test, y_pred)
conf_3=confusion_matrix(Y_test, y_pred)

with mlflow.start_run(run_name="Multilayer Perceptron"):

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.sklearn.log_model(clf, "model")
    
    print("\nMultilayer Perceptron")
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr}")
    print(f"Confusion Matrix {conf_3} \n\n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="multilayer-perceptron-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="multilayer-perceptron-model")
    else:
      mlflow.sklearn.log_model(clf, "model")

2023/02/27 10:01:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '00e180b7ad2143f8a7d7f164b8ed8f8e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Multilayer Perceptron
Accuracy: 0.979372197309417
AUCPR: 0.9338278147695188
Confusion Matrix [[959   2]
 [ 21 133]] 




Successfully registered model 'multilayer-perceptron-model'.
2023/02/27 10:02:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: multilayer-perceptron-model, version 1
Created version '1' of model 'multilayer-perceptron-model'.


In [56]:
print(mlflow.tracking.MlflowClient().get_model_version("multilayer-perceptron-model", '1'))

<ModelVersion: creation_timestamp=1677492132181, current_stage='None', description=None, last_updated_timestamp=1677492132181, name='multilayer-perceptron-model', run_id='6ac085c24a5f446bb23fe6c4307b7fee', run_link=None, source='file:///content/mlruns/0/6ac085c24a5f446bb23fe6c4307b7fee/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>


The MLP Classifier performs the best among the three used models.